In [ ]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import random
%matplotlib inline

In [ ]:
# train_label 폴더 경로
train_label_dir = "C:/vscode/flaw classification/train_label"

In [ ]:
LABEL_NAMES = [
    "가구수정",
    "걸레받이수정",
    "곰팡이",
    "꼬임",
    "녹오염",
    "들뜸",
    "면불량",
    "몰딩수정",
    "반점",
    "석고수정",
    "오염",
    "오타공",
    "울음",
    "이음부불량",
    "창틀,문틀수정",
    "터짐",
    "틈새과다",
    "피스",
    "훼손"
]

In [ ]:
# 각 라벨별 파일 개수 계산
file_counts = []
for i in range(19):
    # 라벨에 속하는 폴더 경로
    label_dir = os.path.join(train_label_dir, str(i))
    # 폴더 내의 파일 개수 계산
    file_count = len([f for f in os.listdir(label_dir) if f.endswith('.png')])
    file_counts.append(file_count)

# 막대 그래프로 시각화
x = np.arange(19)
plt.bar(x, file_counts)
plt.xticks(x, LABEL_NAMES, rotation='vertical')
plt.ylabel('Number of Files')
plt.show()

In [ ]:
# 각 라벨에 속하는 이미지 중에서 랜덤으로 3장 보여주기
for i in range(19):
    # 라벨에 속하는 폴더 경로
    label_dir = os.path.join(train_label_dir, str(i))
    # 폴더 내의 이미지 파일 경로
    img_paths = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith('.png')]
    # 라벨 번호와 해당 라벨의 한글 이름 출력
    print("Label {}: {}".format(i, LABEL_NAMES[i]))
    # 랜덤으로 4개의 이미지 선택하여 보여주기
    img_sample = random.sample(img_paths, 3)
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    for k in range(3):
        img = cv2.imread(img_sample[k])
        axs[k].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
def binary_threshold(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary

In [ ]:
def canny_edge_detection(img):
    binary = binary_threshold(img)
    canny = cv2.Canny(binary, 50, 150)
    return canny

def sobel_operator(img):
    binary = binary_threshold(img)
    sobelx = cv2.Sobel(binary, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(binary, cv2.CV_64F, 0, 1, ksize=5)
    sobel = np.hypot(sobelx, sobely)
    sobel = np.uint8(sobel)
    return sobel

def laplacian_of_gaussian(img):
    binary = binary_threshold(img)
    laplacian = cv2.Laplacian(binary, cv2.CV_64F)
    laplacian_8u = np.uint8(np.absolute(laplacian))
    return laplacian_8u

In [ ]:
# 각 라벨에 속하는 이미지 중에서 랜덤으로 3장 보여주기
for i in range(19):
    # 라벨에 속하는 폴더 경로
    label_dir = os.path.join(train_label_dir, str(i))
    # 폴더 내의 이미지 파일 경로
    img_paths = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith('.png')]
    # 라벨 번호와 해당 라벨의 한글 이름 출력
    print("Label {}: {}".format(i, LABEL_NAMES[i]))
    # 랜덤으로 4개의 이미지 선택하여 보여주기
    img_sample = random.sample(img_paths, 3)
    fig, axs = plt.subplots(3, 5, figsize=(25, 10))
    for k in range(3):
        img = cv2.imread(img_sample[k])
        binary = binary_threshold(img)
        canny = canny_edge_detection(img)
        sobel = sobel_operator(img)
        log = laplacian_of_gaussian(img)
        
        axs[k, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[k, 0].set_title('Original Image')
        
        axs[k, 1].imshow(binary, cmap='gray')
        axs[k, 1].set_title('Binary Image')
        
        axs[k, 2].imshow(canny, cmap='gray')
        axs[k, 2].set_title('Canny Edge Detection')
        
        axs[k, 3].imshow(sobel, cmap='gray')
        axs[k, 3].set_title('Sobel Operator')
        
        axs[k, 4].imshow(log, cmap='gray')
        axs[k, 4].set_title('Laplacian of Gaussian')

    for ax in axs.flat:
        ax.label_outer()
    plt.show()
